In [1]:
from MyMatrixWrapper import Matrix
import torch, numpy as np
torch.cuda.is_available()

True

In [2]:
kernel = Matrix.from_ndarray(np.array([[-1,2,-1]], dtype=float))

In [3]:
image = Matrix.from_ndarray(np.arange(10, dtype=float).reshape(2,-1))

In [4]:
image.conv(kernel)

Matrix.from_ndarray(
array([[0., 0., 0.],
       [0., 0., 0.]]))